# Attribution Demo 

<a target="_blank" href="https://colab.research.google.com/github/safety-research/circuit-tracer/blob/main/demos/attribute_demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In this demo, you'll learn how to load models and perform attribution on them.

In [1]:
# @title Colab Setup Environment

try:
    import google.colab

    !mkdir -p repository && cd repository && \
     git clone https://github.com/safety-research/circuit-tracer && \
     curl -LsSf https://astral.sh/uv/install.sh | sh && \
     uv pip install -e circuit-tracer/

    import sys
    from huggingface_hub import notebook_login

    sys.path.append("repository/circuit-tracer")
    sys.path.append("repository/circuit-tracer/demos")
    notebook_login(new_session=False)
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    # import sys
    # sys.path.insert(0, "/home/tu/circuit-tracer/circuit_tracer")

In [2]:
from pathlib import Path
import torch

from circuit_tracer import ReplacementModel, attribute
from circuit_tracer.utils.create_graph_files import create_graph_files  
from circuit_tracer.graph import Graph, prune_graph, compute_graph_scores

First, load your model and transcoders by name. `model_name` is a normal HuggingFace / [TransformerLens](https://github.com/TransformerLensOrg/TransformerLens) model name; we'll use `google/gemma-2-2b`. We set `transcoder_name` to `gemma`, which is shorthand for the [Gemma Scope](https://arxiv.org/abs/2408.05147) transcoders; we take the transcoders with lowest L0 (mean # of active features) for each layer.

We additionally support `model_name = "meta-llama/Llama-3.2-1B"`, with `"llama"` transcoders; these are ReLU skip-transcoders that we trained, available [here](https://huggingface.co/mntss/skip-transcoder-Llama-3.2-1B-131k-nobos/tree/new-training).

If you want to use other models, you'll have to provide your own transcoders. To do this, set `transcoder_name` to point to your own configuration file, specifying the list of transcoders that you want to use. You can see `circuit_tracer/configs` for example configs.

In [ ]:
model_name = "google/gemma-2-2b" # google/gemma-scope-2-4b-it crosscoder/layer_9_17_22_29_width_65k_l0_medium
transcoder_name = "mntss/clt-gemma-2-2b-2.5M"
backend = 'transformerlens'  # change to 'nnsight' for the nnsight backend!
model = ReplacementModel.from_pretrained(
    model_name, transcoder_name, dtype=torch.bfloat16, backend=backend
)

config.yaml:   0%|          | 0.00/142 [00:00<?, ?B/s]

Fetching 52 files:   0%|          | 0/52 [00:00<?, ?it/s]

W_dec_12.safetensors:   0%|          | 0.00/6.34G [00:00<?, ?B/s]

W_dec_15.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

W_dec_1.safetensors:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

W_dec_10.safetensors:   0%|          | 0.00/7.25G [00:00<?, ?B/s]

W_dec_14.safetensors:   0%|          | 0.00/5.44G [00:00<?, ?B/s]

W_dec_0.safetensors:   0%|          | 0.00/11.8G [00:00<?, ?B/s]

W_dec_11.safetensors:   0%|          | 0.00/6.79G [00:00<?, ?B/s]

W_dec_13.safetensors:   0%|          | 0.00/5.89G [00:00<?, ?B/s]

In [6]:
model_name = "meta-llama/Llama-3.2-1B" # google/gemma-scope-2-4b-it crosscoder/layer_9_17_22_29_width_65k_l0_medium
transcoder_name = "mntss/clt-llama-3.2-1b-524k"
backend = 'transformerlens'  # change to 'nnsight' for the nnsight backend!
model = ReplacementModel.from_pretrained(
    model_name, transcoder_name, dtype=torch.bfloat16, backend=backend
)

Fetching 32 files:   0%|          | 0/32 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loaded pretrained model meta-llama/Llama-3.2-1B into HookedTransformer


Next, set your attribution arguments.

In [4]:
prompt = "Fact: The capital of the state containing Dallas is"  # What you want to get the graph for
max_n_logits = 10  # How many logits to attribute from, max. We attribute to min(max_n_logits, n_logits_to_reach_desired_log_prob); see below for the latter
desired_logit_prob = 0.95  # Attribution will attribute from the minimum number of logits needed to reach this probability mass (or max_n_logits, whichever is lower)
max_feature_nodes = 8192  # Only attribute from this number of feature nodes, max. Lower is faster, but you will lose more of the graph. None means no limit.
batch_size = 256  # Batch size when attributing
offload = (
    "disk" if IN_COLAB else "cpu"
)  # Offload various parts of the model during attribution to save memory. Can be 'disk', 'cpu', or None (keep on GPU)
verbose = True  # Whether to display a tqdm progress bar and timing report

Then, just run attribution!

In [5]:
graph = attribute(
    prompt=prompt,
    model=model,
    max_n_logits=max_n_logits,
    desired_logit_prob=desired_logit_prob,
    batch_size=batch_size,
    max_feature_nodes=max_feature_nodes,
    offload=offload,
    verbose=verbose,
)

Phase 0: Precomputing activations and vectors
Precomputation completed in 0.62s
Found 2029 active features
Phase 1: Running forward pass
Forward pass completed in 0.05s
Phase 2: Building input vectors
Selected 10 logits with cumulative probability 0.5547
Will include 2029 of 2029 feature nodes
Input vectors built in 1.25s
Phase 3: Computing logit attributions
<sys>:0: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
Logit attributions completed in 0.10s
Phase 4: Computing feature attributions
Feature influence computation: 100%|██████████| 2029/2029 [00:00<00:00, 10230.44it/s]
Feature attributions completed in 0.20s
Attribution completed in 6.41s


We now have a graph object! We can save it as a .pt file, but be warned that it's large (~167MB).

In [3]:
graph_dir = "graphs"
graph_name = "austin.pt"
graph_dir = Path(graph_dir)
graph_dir.mkdir(exist_ok=True)
graph_path = graph_dir / graph_name

graph = Graph.from_pt(graph_path)  # Load from disk if it exists, otherwise compute and save to disk
# graph.to_pt(graph_path)

In [3]:
graph_json = "temp_graph_files/puppy-clt.json"
graph = Graph.from_json(graph_json)  # Load from a .json file (e.g. from the tracer output)

TypeError: object of type 'NoneType' has no len()

In [8]:
len(graph.active_features), len(graph.selected_features)

(8296, 8192)

In [9]:
graph.activation_values

tensor([ 1.0703,  1.1641,  1.8516,  ..., 10.5000, 37.2500, 10.4375],
       dtype=torch.bfloat16)

Given this object, we can create the graph files that we need to visualize the graph. Give it a slug (name), and set the node / edge thresholds for pruning. Pruning removes unimportant nodes and edges from your graph; lower thresholds (i.e., more aggressive pruning) results in smaller graphs. These may be easier to interpret, but explain less of the model's behavior.

In [27]:
slug = "dallas-austin-llamma"  # this is the name that you assign to the graph
graph_file_dir = "./graph_files"  # where to write the graph files. no need to make this one; create_graph_files does that for you
node_influence_threshold = 0.5  # keep only the minimum # of nodes whose cumulative influence is >= 0.8
edge_influence_threshold = 0.8  # keep only the minimum # of edges whose cumulative influence is >= 0.98

create_graph_files(
    graph_or_path=graph_path,  # the graph to create files for
    slug=slug,
    output_path=graph_file_dir,
    token_weights = None,
    logit_weights = None,
    node_influence_threshold=node_influence_threshold,
    edge_influence_threshold=edge_influence_threshold,
    node_relevance_threshold=0.5,
    edge_relevance_threshold=0.8,
    keep_all_tokens_and_logits=True,
)

Now, you can visualize the graph using the following commands! This will spin up a local server to act as the frontend.

**If you're running this notebook on a remote server, make sure that you set up port forwarding, so that the chosen port is accessible on your local machine too.**

You can select nodes by clicking on them. Ctrl/Cmd+Click on nodes to pin and unpin them to your subgraph. G+Click on nodes in the subgraph to group them together into a supernode; G+Click on the X next to a supernode to dissolve it. Click on the edit button to edit node descriptions, and click on supernode description to edit that.

In [29]:
from circuit_tracer.frontend.local_server import serve

port = 8046
server = serve(data_dir="./graph_files/", port=port)

if IN_COLAB:
    from google.colab import output as colab_output  # noqa

    colab_output.serve_kernel_port_as_iframe(
        port, path="/index.html", height="800px", cache_in_notebook=True
    )
else:
    from IPython.display import IFrame

    print(f"Use the IFrame below, or open your graph here: f'http://localhost:{port}/index.html'")
    display(IFrame(src=f"http://localhost:{port}/index.html", width="100%", height="800px"))

Use the IFrame below, or open your graph here: f'http://localhost:8046/index.html'


Once you're done, you can stop the server with the following command.

In [28]:
server.stop()

Congrats, you're done! Go to `intervention_demo.ipynb` to see how to perform interventions, or check out `gemma_demo.ipynb` and `llama_demo.ipynb` for examples of worked-out test examples. Read on for a bit more info aabout the Graph class and pruning.

## Graphs

Earlier, you created a graph object. Its adjacency matrix / edge weights are stored in `graph.adjacency_matrix` in a dense format; rows are target nodes and columns are source nodes. The first `len(graph.real_features)` entries of the matrix represent features; the `i`th entry corresponds to the `i`th feature in `graph.real_features`, given in `(layer, position, feature_idx)` format. The next `graph.cfg.n_layers * graph.n_pos` entries are error_nodes. The next `graph.n_pos` entries are token nodes. The final `len(graph.logit_tokens)` entries are logit nodes.

The value of the cell `graph.adjacency_matrix[target, source]` is the direct effect of the source node on the target node. That is, it tells you how much the target node's value would change if the source node were set to 0, while holding the attention patterns, layernorm denominators, and other feature activations constatnt. Thus, if the target node is a feature, this tells you how much the target feature would change; if the target node is a logit, this tells you how much the (de-meaned) value of the logit would change.

Note that `gemma-2-2b` is model (family) that uses logit softcapping. This means that a softcap function, `softcap(x) = t * tanh(x/t)` is used to constrain the logits to fall within (-t, t); `gemma-2-2b` uses `t=30`. For such models, we predict the change in logits *pre-softcap*, as the nonlinearity introduced by softcapping would cause our attribution to yield incorrect / approximate direct effect values.

### Pruning
Given a graph, you might want to prune it, as it will otherwise contain many low-impact nodes and edges that clutter the circuit diagram while adding little information. We enable you to prune nodes by absolute influence, i.e. the total impact that the nodes have on the logits, direct and indirect. The default threshold is 0.8: this means we will keep the minimum number of nodes required to capture 80% of all logit effects. Similarly, the edge_threshold, by default 0.98, means that we will keep the minimum number of edges required to capture 98% of all logit effects.

# Visualization

In [10]:
from circuit_tracer.graph import prune_graph

prune_result = prune_graph(graph, token_weights=None, logit_weights=None, 
            node_influence_threshold=0.5, edge_influence_threshold=0.8,
            node_relevance_threshold=0.7, edge_relevance_threshold=0.8,
            keep_all_tokens_and_logits=True
            )

In [5]:
n_total = graph.adjacency_matrix.shape[0]
n_logits = len(graph.logit_tokens)
n_tokens = len(graph.input_tokens)
n_features = len(graph.selected_features)

In [6]:
print(n_total, n_logits, n_tokens, n_features)

8472 10 10 8192


In [22]:
graph.logit_probabilities.sum()

tensor(0.7148, dtype=torch.bfloat16)

In [11]:
# influence scores of tokens and logits
prune_result[0].sum()

tensor(78)

In [24]:
# relevance scores of tokens and logits
num_ones = int(torch.isclose(prune_result[3], torch.tensor(1.0)).sum().item())
print(num_ones, prune_result[3].shape)

275 torch.Size([8472])


In [10]:
num_ones = int(torch.isclose(prune_result[2], torch.tensor(1.0)).sum().item())
print(num_ones, prune_result[2].shape)

62 torch.Size([2226])


In [13]:
prune_result[3][prune_result[0]]

tensor([0.6189, 0.6748, 0.6926, 0.6312, 0.5516, 0.5262, 0.5866, 0.6751, 0.5050,
        0.6277, 0.4985, 0.4699, 0.4324, 0.4513, 0.5936, 0.4409, 0.4994, 0.4260,
        0.5325, 0.4373, 0.3927, 0.4004, 0.4233, 0.4093, 0.4568, 0.3434, 0.3632,
        0.5346, 0.3811, 0.4336, 0.3536, 0.3594, 0.4620, 0.4524, 0.4136, 0.4151,
        0.3759, 0.3878, 0.4444, 0.3455, 0.3989, 0.3943, 0.3777, 0.2713, 0.3103,
        0.3257, 0.3048, 0.4361, 0.3348, 0.3911, 0.2020, 0.2514, 0.2815, 0.2406,
        0.2159, 0.1939, 0.2563, 0.2293, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.1115, 0.0950, 0.1558, 0.1692,
        0.0403, 0.0212, 0.1419, 0.1818, 0.0772, 0.1274])

In [ ]:
from graph_visualization import visualize_intervention_graph

visualize_intervention_graph(merged_G, prompt = prompt)